# Interactive 3D visualization of Fastscape model outputs

Model outputs can be viewed interactively in 3-d rendered scenes using [ipyfastscape](https://github.com/fastscape-lem/ipyfastscape)'s `TopoViz3d`, very much like [Paraview](https://www.paraview.org/) but in Jupyter notebooks.

In [1]:
import numpy as np
import xsimlab as xs
import xarray as xr

from fastscape.models import basic_model
from ipyfastscape import TopoViz3d

Let's start by setting and running a simple fastscape simulation. Model outputs will be saved along the `time` dimension (clock).

In [2]:
in_ds = xs.create_setup(
    model=basic_model,
    clocks={
        'clock': np.arange(0, 2e6 + 2e4, 2e4),
        'time': np.arange(0, 2e6 + 2e4, 2e4),
    },
    master_clock='time',
    input_vars={
        'grid__shape': [201, 201],
        'grid__length': [2e5, 2e5],
        'boundary__status': 'fixed_value',
        'uplift__rate': 1e-3,
        'spl': {
            'k_coef': 1e-6,
            'area_exp': 0.6,
            'slope_exp': 1
        },
        'diffusion__diffusivity': 1e-1
    },
    output_vars={
        'topography__elevation': 'time',
        'drainage__area': 'time',
        'spl__chi': 'time'
    }
)

In [3]:
with xs.monitoring.ProgressBar():
    out_ds = in_ds.xsimlab.run(model=basic_model)

In [4]:
out_ds

<xarray.Dataset>
Dimensions:                 (border: 4, clock: 101, shape_yx: 2, time: 101, x: 201, y: 201)
Coordinates:
  * border                  (border) <U6 'left' 'right' 'top' 'bottom'
  * clock                   (clock) float64 0.0 2e+04 4e+04 ... 1.98e+06 2e+06
  * time                    (time) float64 0.0 2e+04 4e+04 ... 1.98e+06 2e+06
  * x                       (x) float64 0.0 1e+03 2e+03 ... 1.99e+05 2e+05
  * y                       (y) float64 0.0 1e+03 2e+03 ... 1.99e+05 2e+05
Dimensions without coordinates: shape_yx
Data variables:
    boundary__status        <U11 'fixed_value'
    diffusion__diffusivity  float64 0.1
    drainage__area          (time, y, x) float64 1e+06 1.1e+07 ... 2e+06 1e+06
    grid__length            (shape_yx) float64 2e+05 2e+05
    grid__shape             (shape_yx) int64 201 201
    spl__area_exp           float64 0.6
    spl__chi                (time, y, x) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    spl__k_coef             float64 1e-06
    spl__slope_exp          int64 1
    topography__elevation   (time, y, x) float64 0.6748 0.9104 ... 0.4401 0.2293
    uplift__rate            float64 0.001

The output dataset `out_ds` can be readily used with `TopoViz3d`, which creates a graphical interface with the 3-d rendered scene as well as convenient controls for some display properties. Here we also explicitly set `time="time"` to enable animation of the 3D scene using some additional controls.

In [5]:
v1 = TopoViz3d(out_ds, canvas_height=600, time="time")

v1.show()

Output(layout=Layout(height='640px'))

It is also possible to control the visualization from the notebook code cells. For example, let's show the time step that is the closest to a given time value (you should see the output of the cell here above updated accordingly):

In [7]:
v1.timestepper.go_to_time(1.15e6)